In [1]:
import time
import pandas as pd
import numpy as np
import scipy.sparse as sps
import random as rnd

from scipy.sparse import *

In [2]:
urm_path = './content/data_train.csv'
urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]
print ("The number of interactions is {}".format(len(urm_all_df)))

The number of interactions is 478730


In [3]:
URM_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

URM_all

<13025x22348 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in COOrdinate format>

In [4]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2978 (22.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2582 (19.8%) Users that have less than 1 test interactions


In [ ]:
#!pip install lightfm

In [5]:
# Set the number of threads; you can increase this
# if you have more physical cores available.
from Recommenders.FactorizationMachines.LightFMRecommender import LightFMCFRecommender
recommender = LightFMCFRecommender(URM_train)


LightFMCFRecommender: URM Detected 810 ( 6.2%) users with no interactions.
LightFMCFRecommender: URM Detected 465 ( 2.1%) items with no interactions.


d:\Anaconda\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [6]:
recommender.fit(epochs = 10)

: 

In [ ]:
result_df, _ = evaluator_validation.evaluateRecommender(recommender)
result_df